In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer, MultiLabelBinarizer, scale, minmax_scale
from scipy.spatial.distance import cosine
from scipy import sparse
import lightgbm as lgb
import time
import multiprocessing
%matplotlib inline

In [77]:
data_df = pd.read_csv('data_df_new.csv',index_col=0)

/home/sc3/miniconda2/envs/ydzhang/lib/python3.6/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
def count_len(text):
    if text=='-1':
        return 0
    return len(text.split(','))
def get_day(str):
    return int(str.split('-')[0][1:])
def get_hour(str):
    return int(str.split('-')[1][1:])
def split_topic(str):
    if str == '-1':
        return []
    topics = str.split(',')
    for i in range(len(topics)):
        topics[i] = int(topics[i][1:])
    return topics

def split_interest(str):
    if(str)=='-1':
        return {};
    re = str.split(',')
    red = {}
    for i in re:
        i = i.split(':')
        val = float(i[1])
        if(val==float('inf')):
            val = 10
        red[int(i[0][1:])] = val
    return red

def topic_sim(follow,interest,question):
    sq = set(question); sf = set(follow) ;si = set(interest.keys())
    sqsf = sq&sf ; sqsi = sq&si
    sqf = sq | sf; sqi = sq | si
    sqfi = sqf | sqi
    val = 0
    max_in = 0
    for t in sqsi:
        val += interest[t]
        if interest[t] > max_in:
            max_in = interest[t]
    return np.array([val,max_in,len(sqsf),len(sqsi),len(sqf),len(sqi),len(sqfi)])


In [4]:
# data_dir = '../data/zhihu competition/'
# member_df = pd.read_csv(os.path.join(data_dir, 'member_info_0926.txt'), header=None, sep='\t')
# member_df.columns = ['user_id', 'gender', 'keywords', 'creation_num_level', 'creation_heat_level', 'register_type', 'register_platform',
#                'visit_freq', 'binary_A', 'binary_B', 'binary_C', 'binary_D', 'binary_E', 'category_A', 'category_B', 'category_C', 
#                'category_D', 'category_E', 'salt_value', 'follow_topics', 'interest_topics']
# member_df.to_csv('member_info.csv')
member_df = pd.read_csv('member_info.csv',index_col=0,header=0)
encoder = {}
encoder['gender'] = LabelEncoder()
member_df['gender_label'] = encoder['gender'].fit_transform(member_df['gender'])

visit_label = dict([['unknown',-1],['new',0],['monthly',1],['daily',2],['weekly',3]])
member_df['visit_freq_label'] = member_df['visit_freq'].apply(lambda x:visit_label[x])

multi_sectors = ['category_A','category_B','category_C','category_D','category_E']
for sec in multi_sectors:
    encoder[sec+'_label'] = LabelEncoder()
    member_df[sec + '_label'] = encoder[sec+'_label'].fit_transform(member_df[sec])

In [5]:
# question_columns = ['question_id','create_time', 'title_SW', 'title_W','describe_SW','describe_W','topic_id']
# question_df = pd.read_csv(os.path.join(data_dir, 'question_info_0926.txt'),
#                           nrows = None,sep= '\t',names = question_columns)
# question_df.to_csv('question_info.csv')
question_df = pd.read_csv('question_info.csv',index_col=0)
question_df['create_day'] = question_df['create_time'].apply(get_day)
question_df['create_hour'] = question_df['create_time'].apply(get_hour)
question_df['title_len'] = question_df['title_SW'].apply(count_len)
question_df['describe_len'] = question_df['describe_SW'].apply(count_len)

In [ ]:
# data_df = pd.merge(data_df,member_df[['user_id','user_id_count_userdf']],on = 'user_id',how='left')
# data_df = pd.merge(data_df,question_df[['question_id','question_id_count_questiondf']], on = 'question_id', how = 'left')
# for sec in ['user_id_count_userdf','question_id_count_questiondf']:
#     data_df[sec] = minmax_scale(data_df[sec])

In [ ]:
# data_dir = '../data/zhihu competition/'
# ans_columns = ['answer_id', 'question_id', 'user_id', 'create_time', 'answer_SW', 'answer_W',
#                  'is_good', 'is_recommend', 'is_include', 'has_picture', 'has_video', 'word_count', 'num_agree','num_cancel_angree',
#                  'num_comment', 'num_favorite', 'num_thanks', 'num_report', 'num_useless', 'num_oppose']
# answer_df = pd.read_csv(os.path.join(data_dir, 'answer_info_0926.txt'), sep= '\t', nrows = None, names = ans_columns)
# answer_df.to_csv('answer_info.csv')
answer_df = pd.read_csv('answer_info.csv',index_col=0)
answer_df['answer_create_day'] = answer_df['create_time'].apply(get_day)
answer_df['answer_create_hour'] = answer_df['create_time'].apply(get_hour)
answer_df = pd.merge(answer_df,question_df[['question_id','title_len','describe_len']],how='left',on='question_id')

In [ ]:
trian_start = 3838; train_end = 3867
test_start = 3868; test_end = 3874
ans_start = 3807; ans_end = 3867
ques_start = 753;qus_end = 3874 # question 暂时不用处理
ans_numeric_secs = ['answer_create_day','is_good','has_picture', 'has_video', 'word_count', 'num_agree','num_cancel_angree','num_comment', 'num_favorite', 'num_thanks', 'num_report', 'num_useless', 'num_oppose']
# ans_numeric_secs = ['word_count', 'num_agree', 'answer_create_day']
def get_ans_data(answer_df):
    member_ans_df = answer_df[['user_id','title_len','describe_len']+ans_numeric_secs].groupby('user_id').mean()
    member_ans_df.reset_index(inplace=True)
    member_ans_df['user_answer_nums'] = member_ans_df['user_id'].map(answer_df['user_id'].value_counts())
    member_ans_df.columns = [i+'_user' if i[0:4]!= 'user' else i for i in member_ans_df.columns ]

    question_ans_df = answer_df[['question_id']+ans_numeric_secs].groupby('question_id').mean()
    question_ans_df.reset_index(inplace=True)
    question_ans_df['question_answer_nums'] = question_ans_df['question_id'].map(answer_df['question_id'].value_counts())
    question_ans_df.columns = [i+'_question' if i[0:8]!= 'question' else i for i in question_ans_df.columns ]
    return member_ans_df,question_ans_df

In [ ]:
# NA数据未处理
test_member_ans_df,test_question_ans_df = get_ans_data(answer_df)
train_member_ans_df,train_question_ans_df = get_ans_data(answer_df[answer_df['answer_create_day']<train_end-7])

In [ ]:
# train_df = pd.read_csv(os.path.join(data_dir, 'invite_info_0926.txt'), header = None, names = ['question_id', 'user_id', 'create_time','is_answer'],
#                 sep= '\t')
# train_df.to_csv('invite_info.csv')
# test_df = pd.read_csv(os.path.join(data_dir, 'invite_info_evaluate_1_0926.txt'), header = None, names = ['question_id', 'user_id', 'create_time'],
#                 sep= '\t')
# test_df.to_csv('evalute.csv')
train_df =pd.read_csv('invite_info.csv',index_col=0)
test_df =pd.read_csv('evalute.csv',index_col=0)
train_df['invite_create_day'] = train_df['create_time'].apply(get_day)

In [ ]:
train_ans_df = pd.merge(train_df,train_member_ans_df,how='left', on='user_id')
train_ans_df = pd.merge(train_ans_df,train_question_ans_df,how='left', on='question_id')
test_ans_df = pd.merge(test_df,test_member_ans_df,how='left', on='user_id')
test_ans_df = pd.merge(test_ans_df,test_question_ans_df,how='left', on='question_id')

train_df['invite_create_day'] = train_ans_df['create_time'].apply(get_day)
test_ans_df['invite_create_day'] = test_df['create_time'].apply(get_day)

In [ ]:
invite_df = pd.concat([train_ans_df,test_ans_df],axis=0,sort=True)
invite_df['invite_create_day'] = invite_df['create_time'].apply(get_day)
invite_df['invite_create_hour'] = invite_df['create_time'].apply(get_hour)
invite_df.drop(columns=['create_time'],inplace=True)

member_data = member_df[['user_id','salt_value','gender_label', 'visit_freq_label', 'binary_A', 'binary_B', 'binary_C',
       'binary_D', 'binary_E', 'category_A_label', 'category_B_label','category_C_label', 'category_D_label', 'category_E_label',
                        'follow_topics', 'interest_topics']]
question_data = question_df[['question_id','create_day','create_hour','title_len','describe_len','topic_id']]

#合并数据
data_df = pd.merge(invite_df,question_data,on='question_id',how='left')
data_df = pd.merge(data_df,member_data,on='user_id',how='left')
# 处理id
for sec in ['question_id', 'user_id']:
    encoder[sec+'_label'] = LabelEncoder()
    data_df[sec + '_label'] = encoder[sec+'_label'].fit_transform(data_df[sec])

#邀请和问题创建时间的间隔
data_df['question_answer_day'] = data_df['invite_create_day'] - data_df['create_day']  
    
#统计频数
for feat_name in [ 'category_A_label', 'category_B_label','category_C_label', 'category_D_label', 'category_E_label','gender_label', 'visit_freq_label', 'binary_A', 'binary_B', 'binary_C',
       'binary_D', 'binary_E','question_id', 'user_id']:
    sec = feat_name+'count'
    data_df[sec] = data_df[feat_name].map(data_df[feat_name].value_counts())

In [32]:
question_topics = question_df['topic_id'].apply(split_topic).values.tolist()
# question_topics = data_df['topic_id'].values.tolist()

In [33]:
follow_topics = member_df['follow_topics'].apply(split_topic).values.tolist()

In [65]:
topics_matrix = np.zeros([100000,100000],dtype = 'int32')

In [66]:
for topics in question_topics:
    topics = [x-1 for x in topics]
    for x in topics:
        topics_matrix[x][topics] += 1

In [67]:
topics_matrix = topics_matrix.T

In [68]:
for topics in follow_topics:
    topics = [x-1 for x in topics]
    for x in topics:
        topics_matrix[x][topics] += 1

In [78]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=100, n_iter=5, random_state=42,ve)

In [ ]:
topic_svd_vecs = svd.fit_transform(topics_matrix)

In [69]:
for i in range(100000):
    topics_matrix[i][i] = int(topics_matrix[i][i]/2)

In [86]:
topic_svd_vecs.shape

(100000, 100)

In [100]:
np.save('topic_svd_vecs.npy',topic_svd_vecs)

In [93]:
topic_svd_vecs = np.vstack([np.zeros(100),topic_svd_vecs])

In [95]:
question_svd_vecs = list(map(lambda x:topic_svd_vecs[x].mean(axis=0),question_topics))

/home/sc3/miniconda2/envs/ydzhang/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: Mean of empty slice.
  """Entry point for launching an IPython kernel.
/home/sc3/miniconda2/envs/ydzhang/lib/python3.6/site-packages/numpy/core/_methods.py:154: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


In [96]:
user_svd_vecs = list(map(lambda x:topic_svd_vecs[x].mean(axis=0),follow_topics))

/home/sc3/miniconda2/envs/ydzhang/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: Mean of empty slice.
  """Entry point for launching an IPython kernel.


In [97]:
data_df['topic_svd_dis'] = np.sum(np.square(np.array(user_svd_vecs)-np.array(question_svd_vecs)),axis = 1)

In [99]:
data_df['topic_svd_dis'] = minmax_scale(data_df['topic_svd_dis'])

In [94]:
question_topics = data_df['topic_id'].apply(split_topic).values.tolist()
# question_topics = data_df['topic_id'].values.tolist()

follow_topics = data_df['follow_topics'].apply(split_topic).values.tolist()
# follow_topics = data_df['follow_topics'].values.tolist()

In [ ]:
interest_topics = data_df['interest_topics'].apply(split_interest).values.tolist()

## 基于问题和用户协同过滤

## 基于问题词向量

In [ ]:
ques_index_df = pd.read_csv('question_index.csv',index_col=0)

In [ ]:
ques_vecs = np.load('question_vecs.npy')

In [ ]:
qids_dicts = {}
for uid in answer_df['user_id'].unique():
    qids_dicts[uid] = que_filter_df.get_group(uid).values[:,1].tolist()

In [ ]:
def compare_ques_vecs(items):
    qid,uid = items
    re =  np.array([np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])
    if uid not in uids:
        return re
    else:
        q_vecs = ques_vecs[ques_index_df.loc[qid,'index']]
        qids = qids_dicts[uid]
        try:
            qids.remove(qid)
        except:
            pass
        if(len(qids)==0):
            return re
        q_indexs = ques_index_df.loc[qids,'index'].values
        his_vecs = ques_vecs[q_indexs]
        val = his_vecs - q_vecs
        val = np.sqrt(np.square(val).sum(axis=2))
        re[0:3] = val.mean(axis=0)
        re[3:6] = val.min(axis=0)
    return re

In [ ]:
que_filter_df = answer_df[['user_id','question_id']].groupby('user_id')
uids = answer_df['user_id'].unique().tolist()
all_id = data_df[['question_id','user_id']].values.tolist()

In [ ]:
compare_ques_vecs(all_id[4])

In [28]:
np.save('y_train.npy',y_train)

In [ ]:
import multiprocessing
cores = multiprocessing.cpu_count()-5
pool = multiprocessing.Pool(processes=cores)

In [ ]:
start = time.time()
ques_vec_dis = pool.map(compare_ques_vecs,all_id)
end = time.time()
print(time.time()-start)

In [ ]:
ques_vecs_secs = ['title_SW_vec_his_mean', 'title_W_vec_his_mean','describe_W_vec_his_mean','title_SW_vec_his_max', 'title_W_vec_his_max','describe_W_vec_his_max']
data_df[ques_vecs_secs] = quesVecDis

In [ ]:
for sec in ques_vecs_secs:
    data_df[sec] = minmax_scale(data_df[sec])
#     data_df[sec] = 0

In [ ]:
quesVecDis = [x[0] if type(i)==tuple else x for x in ques_vec_dis[0:10]]

In [ ]:
len(quesVecDis)

In [ ]:
data_df.to_csv('data_df_new.csv')

In [ ]:
data_df[ques_vecs_secs].head()

In [ ]:
print(end-start)

### 基于问题

In [ ]:
question_topics_dict = {}
for _,items in question_df[['question_id','topic_id']].iterrows():
     question_topics_dict[items['question_id']] = split_topic(items['topic_id'])

def jaccard_dis(set1,set2):
    set1 = set(set1); set2 = set(set2)
    return (len(set1&set2))/(len(set1|set2))

def compare_ques_his(items):
    qid,uid = items
    if uid not in uids:
        re =  np.nan,np.nan
    else:
        qids = que_filter_df.get_group(uid).values[:,1].tolist()
        re = []
        set1 = question_topics_dict[qid]
        if(len(set1)==0):
            return np.nan,np.nan
        for qid2 in qids:
            if(qid==qid2):
                continue
            set2 = question_topics_dict[qid2]
            if(len(set2)==0):
                val = np.nan
            else:
                val = jaccard_dis(set1,set2)
            re.append(val)         
        if re==[]:
            re = np.nan,np.nan
        else:
            re = np.array(re)
            re = (re.mean(),re.min())
    return re

que_filter_df = answer_df[['user_id','question_id']].groupby('user_id')
uids = answer_df['user_id'].unique().tolist()
compare_re = []
all_id = data_df[['question_id','user_id']].values.tolist()

import multiprocessing
cores = multiprocessing.cpu_count()-5
pool = multiprocessing.Pool(processes=cores)

start = time.time()
questions_dis = pool.map(compare_ques_his,all_id)
end = time.time()
print(time.time()-start)

In [ ]:
data_df[['his_quesiont_dis_mean', 'his_quesiont_dis_max']] = questions_dis
for sec in ['his_quesiont_dis_mean','his_quesiont_dis_max']:
    data_df[sec] = minmax_scale(data_df[sec])

## 基于用户

In [ ]:
# member_df.shape,question_df.shape

# user_topics_dict = {}
# for _,items in member_df[['user_id','follow_topics']].iterrows():
#      user_topics_dict[items['user_id']] = split_topic(items['follow_topics'])

# uids_dicts = {}
# for uid in answer_df['question_id'].unique():
#     uids_dicts[qid] = que_filter_df.get_group(qid).values[:,1].tolist()

# def jaccard_dis(set1,set2):
#     set1 = set(set1); set2 = set(set2)
#     return (len(set1&set2))/(len(set1|set2))

# def compare_users_his(items):
#     qid,uid = items
# #     if qid not in qids:
#         re =  np.nan,np.nan
#     else:
#         uids = user_filter_df.get_group(qid).values[:,0].tolist()
#         re = []
#         set1 = user_topics_dict[uid]
#         if(len(set1)==0):
#             return np.nan,np.nan
#         for uid2 in uids:
#             if(uid==uid2):
#                 continue
#             set2 = user_topics_dict[uid2]
#             if(len(set2)==0):
#                 val = np.nan
#             else:
#                 val = jaccard_dis(set1,set2)
#             re.append(val)         
#         if re==[]:
#             re = np.nan,np.nan
#         else:
#             re = np.array(re)
#             re = (re.mean(),re.min())
#     return re

# user_filter_df = answer_df[['user_id','question_id']].groupby('question_id')
# qids = answer_df['question_id'].unique().tolist()
# compare_re = []
# all_id = data_df[['question_id','user_id']].values.tolist()

# import time

# import multiprocessing
# cores = multiprocessing.cpu_count()-5
# pool = multiprocessing.Pool(processes=cores)

# start = time.time()
# users_dis = pool.map(compare_users_his,all_id[0:10000])
# end = time.time()
# print(time.time()-start)

# print(end-start)

# data_df[['his_user_dis_mean', 'his_user_dis_max']] = users_dis
# for sec in ['his_user_dis_mean','his_user_dis_max']:
#     data_df[sec] = minmax_scale(data_df[sec])

In [ ]:
# user question 的共现矩阵
# question topic_id 的共现矩阵

# topic 处理 
# 同一个问题下的所有topic，算一类，看能划分出多少并查集
# 计算每个用户关注的话题，兴趣话题 问题的话题的topic的平均值，求相似度 注意用余弦距离

In [ ]:
# data_dir = '../data/zhihu competition/'
# with open(data_dir + 'topic_vectors_64d.txt') as f:
#     lines = f.readlines()
# lines = [i.strip().split() for i in lines]
# topic_vecs = np.array(lines)
# topic_vecs = topic_vecs[:,1:].astype('float32')
# topic_vecs = np.vstack([np.zeros([1,64]),topic_vecs])
# np.save('topic_vecs.npy',topic_vecs)

topic_vecs = np.load('topic_vecs.npy')

question_vecs = list(map(lambda x:topic_vecs[x].mean(axis=0),question_topics))
user_vecs = list(map(lambda x:topic_vecs[x].mean(axis=0),follow_topics))

from scipy.spatial.distance import cosine

data_df['topic_cosdis'] = list(map(cosine,user_vecs,question_vecs))
data_df['topic_eucliddis'] = np.sum(np.square(np.array(user_vecs)-np.array(question_vecs)),axis = 1)

In [ ]:
# del user_vecs, question_vecs

In [ ]:
topic_secs = ['total_interest','max_interest','que&fol','que&inter','que|fol','que|inter','que|fol|inter']
topic_inter = np.zeros([data_df.shape[0],len(topic_secs)])
for i in range(data_df.shape[0]):
    topic_inter[i] = topic_sim(follow_topics[i],interest_topics[i],question_topics[i])

In [ ]:
data_df['qt_len'] = [len(i) for i in question_topics]
data_df['follow_len'] = [len(i) for i in follow_topics]
for i in range(len(topic_secs)):
    data_df[topic_secs[i]] = topic_inter[:,i]
data_df['topic_dis'] = data_df['que&fol'] / data_df['que|fol']

## 重新计算天数

In [18]:
# test_member_ans_df,test_question_ans_df = get_ans_data(answer_df)
dd = 14
train_member_ans_df,train_question_ans_df = get_ans_data(answer_df[answer_df['answer_create_day']<=train_end-dd])

train_ans_df = pd.merge(train_df,train_member_ans_df,how='left', on='user_id')
train_ans_df = pd.merge(train_ans_df,train_question_ans_df,how='left', on='question_id')

update_columns = train_ans_df.columns.drop(['question_id','user_id','is_answer','create_time'])

data_df.loc[0:train_df.shape[0]-1,update_columns] = train_ans_df[update_columns] #.loc()左闭右闭
for sec in update_columns:
    data_df.loc[0:train_df.shape[0]-1,sec] = minmax_scale(data_df[sec][:train_df.shape[0]])

In [19]:
prob = {}
train_answer_praba = train_df[train_df['invite_create_day']<train_end-dd][['user_id','is_answer']].groupby('user_id').mean().to_dict()['is_answer']
prob['train_answer_prob'] = train_ans_df['user_id'].map(train_answer_praba)
train_answer_praba = train_df[train_df['invite_create_day']<train_end-dd][['question_id','is_answer']].groupby('question_id').mean().to_dict()['is_answer']
prob['train_question_prob'] = train_ans_df['question_id'].map(train_answer_praba)
data_df.loc[0:train_df.shape[0]-1,'answer_prob'] = prob['train_answer_prob'].values
data_df.loc[0:train_df.shape[0]-1,'question_prob'] = prob['train_question_prob'].values

In [ ]:
test_answer_praba = train_df[['user_id','is_answer']].groupby('user_id').mean().to_dict()['is_answer']
prob['test_answer_prob'] = test_ans_df['user_id'].map(test_answer_praba)
test_answer_praba = train_df[['question_id','is_answer']].groupby('question_id').mean().to_dict()['is_answer']
prob['test_question_prob'] = test_ans_df['question_id'].map(test_answer_praba)

data_df.loc[train_df.shape[0]:,'answer_prob'] = prob['test_answer_prob'].values
data_df.loc[train_df.shape[0]:,'question_prob'] = prob['test_question_prob'].values

In [ ]:
# data_df.to_csv('data_df.csv')

In [20]:
x_train = data_df[train_secs][:train_df.shape[0]][(train_df['invite_create_day']>train_end-dd) & (train_df['invite_create_day']<=train_end-dd+7)].values
y_train = data_df['is_answer'][:train_df.shape[0]][(train_df['invite_create_day']>train_end-dd) & (train_df['invite_create_day']<=train_end-dd+7)].values

NameError: name 'train_secs' is not defined

In [62]:
x_train5.shape

(1847807, 88)

In [38]:
x_train = np.vstack([x_train7,x_train14])

In [42]:
y_train = np.hstack([y_train7,y_train14])

In [16]:
y_train.shape,x_train.shape

NameError: name 'y_train' is not defined

In [ ]:
data_df.loc[train_df.shape[0]:,'answer_prob'].shape,prob['test_answer_prob'].shape

In [104]:
drop_columns = ['is_answer', 'question_id', 'user_id', 'topic_id','follow_topics', 'interest_topics']
train_secs = data_df.columns.drop(drop_columns)
category_columns = ['user_id_label','question_id_label','category_A_label', 'category_B_label','category_C_label', 'category_D_label', 'category_E_label','gender_label', 'visit_freq_label', 'binary_A', 'binary_B', 'binary_C','binary_D', 'binary_E']
scale_columns = train_secs.drop(category_columns)

In [ ]:
# 归一化数据
for sec in train_secs:
    data_df[sec] = minmax_scale(data_df[sec])

In [ ]:
train_secs = data_df.columns.drop(drop_columns)
ques_vecs_secs

In [105]:
x_train = data_df[train_secs][:train_df.shape[0]][train_df['invite_create_day']>train_end-7].values
y_train = data_df[:train_df.shape[0]]['is_answer'][train_df['invite_create_day']>train_end-7].values
x_test = data_df[train_secs][train_df.shape[0]:].values

In [48]:
data_df.shape,x_train.shape,len(train_secs)

((10630845, 94), (4837532, 88), 88)

In [106]:
import lightgbm as lgb
params = {
    'boosting_type':'gbdt','objective':'binary','metric':['auc', 'binary_logloss'],
    'n_jobs':-1,'num_threads':30,'n_estimators':2000,'subsample':0.8,'subsample_freq':1,
    'num_leaves':64,'learning_rate':0.01,
    'max_bin':425,'subsample_for_bin':50000,'min_split_gain':0,
    'min_child_weight':5,'min_child_samples':10,
    'colsample_bytree':1,'reg_alpha':3,'reg_lambda':5,
    'seed':1000,'silent':True    
}
lgb_train = lgb.Dataset(x_train,y_train,free_raw_data=False, )#, feature_name=train_secs.to_list(), categorical_feature=category_columns)
gbm = lgb.train(params,lgb_train,valid_sets=lgb_train,early_stopping_rounds=5)

[1]	training's auc: 0.779217	training's binary_logloss: 0.438097
Training until validation scores don't improve for 5 rounds
[2]	training's auc: 0.78682	training's binary_logloss: 0.436541
[3]	training's auc: 0.786009	training's binary_logloss: 0.435046
[4]	training's auc: 0.78548	training's binary_logloss: 0.433587
[5]	training's auc: 0.786686	training's binary_logloss: 0.432176
[6]	training's auc: 0.788371	training's binary_logloss: 0.430783
[7]	training's auc: 0.789758	training's binary_logloss: 0.429441
[8]	training's auc: 0.791125	training's binary_logloss: 0.428129
[9]	training's auc: 0.791888	training's binary_logloss: 0.42685
[10]	training's auc: 0.792178	training's binary_logloss: 0.425605
[11]	training's auc: 0.79283	training's binary_logloss: 0.424378
[12]	training's auc: 0.793235	training's binary_logloss: 0.423192
[13]	training's auc: 0.79368	training's binary_logloss: 0.422028
[14]	training's auc: 0.793712	training's binary_logloss: 0.42089
[15]	training's auc: 0.793747	t

In [ ]:
gbm.best_score

In [33]:
gbm.save_model('model7_r.txt')

In [ ]:
# boost = lgb.Booster(model_file='base_model.txt')
# y_pred = boost.predict(x_test)
# test_df['is_answer'] = y_pred
# test_df.to_csv('result.txt', index=False, header=False, sep='\t')

In [107]:
y_pred = gbm.predict(x_test)
test_df['is_answer'] = y_pred
test_df.to_csv('result.txt', index=False, header=False, sep='\t')

In [ ]:
gbm.feature_importance()

In [108]:
sorted(list(zip(train_secs.to_list(),gbm.feature_importance())),key=lambda x:x[1],reverse=True)

[('salt_value', 11473),
 ('question_idcount', 11454),
 ('user_idcount', 9945),
 ('answer_prob', 7779),
 ('invite_create_hour', 6649),
 ('user_answer_nums', 5319),
 ('question_answer_day', 5176),
 ('answer_create_day_user', 4514),
 ('invite_create_day', 3488),
 ('create_day', 3140),
 ('title_W_vec_his_max', 2653),
 ('describe_len', 2638),
 ('create_hour', 2566),
 ('topic_eucliddis', 2419),
 ('his_quesiont_dis_mean', 2331),
 ('title_SW_vec_his_max', 2318),
 ('question_prob', 2250),
 ('title_W_vec_his_mean', 1833),
 ('word_count_user', 1736),
 ('qt_len', 1627),
 ('his_quesiont_dis_max', 1615),
 ('topic_svd_dis', 1608),
 ('topic_cosdis', 1601),
 ('que|inter', 1482),
 ('num_agree_user', 1367),
 ('describe_W_vec_his_max', 1358),
 ('title_len', 1321),
 ('describe_len_user', 1301),
 ('title_len_user', 1258),
 ('question_answer_nums', 1230),
 ('que|fol|inter', 1202),
 ('num_thanks_user', 1199),
 ('title_SW_vec_his_mean', 1191),
 ('describe_W_vec_his_mean', 1050),
 ('word_count_question', 951),


In [52]:
feat_importances = sorted(list(zip(train_secs.to_list(),gbm.feature_importance())),key=lambda x:x[1],reverse=True)

In [54]:
print([i[0] for i in feat_importances[0:10]])

['question_idcount', 'salt_value', 'user_idcount', 'answer_prob', 'invite_create_hour', 'question_answer_day', 'user_answer_nums', 'invite_create_day', 'answer_create_day_user', 'create_day']


In [ ]:
# 不加入id_count , 0.69
# 加入user_id_count，question_id_count有明显提升,到 0.76
# 数据预归一化处理
# 加入 answer_prob 训练0.29，0.88，测试0.58,过拟合
# 加入 question_count,answer_count 也是过拟合
# 加入 top相关特征 训练0.74-0.79 提交0.77
# 加入回答的统计特征'num_agree_user','word_count_user','num_agree_question','word_count_question','question_answer_nums'，'uesr_answer_nums'，训练0.87 测试0.75
# 加入  均值填充nan 'num_agree_user','word_count_user','user_answer_nums'， 0.76-0.82， 0.46-0.35， 77.01 
# 同上 0均值 77.3
#  0均值 0.771 过拟合    'num_agree_user','word_count_user','user_answer_nums','num_agree_question' train's auc: 0.785558	train's binary_logloss: 0.465657 (0.8465617931917354, 0.33882461713047074)
#	mean   0.75 过拟合 'num_agree_user','word_count_user','num_agree_question','word_count_question' train's auc: 0.81656	train's binary_logloss: 0.465285 (0.8714833442600826, 0.316634370916078)
# mean num_agree_user num_agree_question 0.76

## 
# 计算answer_df时不使用最近10天的数据，训练0.79-0.88 提交79.5
# 加入 answer_prob(-10), 使用（-10）数据 ，训练0.77-0.83 提交80.1
# 使用-10 76-82 79.58
# answer question prob （-10）0.77-0.83 78.45
# answer question prob （-20） 数据-10  0.76-0.83 79.8
# answer question prob （-20） 数据-20  0.78-0.86  79.8
# 加入 新特征 answer(-10) 数据-10 0.77-0.839 78.4
# 加入 新特征 answer(-20) 数据-10 0.77-0.831 80.0
# 去除 answer question 数据 -10 80.55

#retrain 前80.1 retrain后80.4
# 加入 category_columns, 使用默认参数，训练0.75-0.85 79.9
# 使用修改后的参数， 训练0.77-0.83 80.51
# 加入 topic_cosdi -10 同上 80.53 似乎不用加入category_columns
# 加入 topic_cosdis 全部数据 80.22
# 同上 去除 category columns, 训练 0.77-0.83 80.56
# 测试以下参数
# 数据天数 -14 77-84 80.57
# 数据天数 -7 76-83 80.63
# 数据天数 -10 80.64
#，预训练   7+7 1000,2000  80.80
# 7+7+7 2000 80.856
# 10 +7 2000 2000 80.85

# 加入 answer_prob -7 -7 82.454
# 加入 answer_prob -7 82.37
# 加入 answer_prob question  -7 82.40
# 加入 answer_prob question  -7 -7 -7 训练84.75 提交82.58
# 加入欧式距离 未归一化 ，使用category -7 82.21
# 加入欧式距离 归一化 ，-7 82.44
# 加入话题距离 -7 82.61
# 加入 category -7 82.43
# 加 user_df和question_df的idcount -7 82.61
# 加入标题信息 -7 82.71

# gbdt categroy 处理方式
# deepFM spares_feature
# lr 

In [ ]:
y_pred = model_lgb.predict_proba(x_test)[:, 1]
test_df['is_answer'] = y_pred
test_df.to_csv('result.txt', index=False, header=False, sep='\t')

In [ ]:
## 构造特征
# 问题，历史回答用户的信息平均值 
# 用户，历史回答问题的特征
# 

In [ ]:
train_answer_praba = train_df[train_df['invite_create_day']<train_end-10][['user_id','is_answer']].groupby('user_id').mean().to_dict()['is_answer']
data_df['answer_prob'][0:train_df.shape[0]] = data_df['user_id'][0:train_df.shape[0]].map(train_answer_praba)

train_answer_praba = train_df[['user_id','is_answer']].groupby('user_id').mean().to_dict()['is_answer']
data_df['answer_prob'][train_df.shape[0]:] = data_df['user_id'][train_df.shape[0]:].map(train_answer_praba)

In [ ]:
train_answer_praba = train_df[train_df['invite_create_day']<train_end-10][['question_id','is_answer']].groupby('question_id').mean().to_dict()['is_answer']
data_df['question_prob'][0:train_df.shape[0]] = data_df['question_id'][0:train_df.shape[0]].map(train_answer_praba)

train_answer_praba = train_df[['question_id','is_answer']].groupby('question_id').mean().to_dict()['is_answer']
data_df['question_prob'][train_df.shape[0]:] = data_df['question_id'][train_df.shape[0]:].map(train_answer_praba)

# deep-fm

In [67]:
from deepctr_torch.models import DeepFM,wdl
from deepctr_torch.inputs import  SparseFeat, DenseFeat,get_feature_names
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
import torch

In [65]:
boost = lgb.Booster(model_file='base_model.txt')
train_gbdt_feat = boost.predict(x_train,pred_leaf=True)

In [68]:
sparse_features = [str(i) for i in range(1000)]
# sparse_features = []
dense_features = ['question_idcount', 'salt_value', 'user_idcount', 'answer_prob', 'invite_create_hour', 'question_answer_day', 'user_answer_nums', 'invite_create_day', 'answer_create_day_user', 'create_day']
dense_pos = [train_secs.get_loc(sec) for sec in dense_features]
fixlen_feature_columns = [SparseFeat(feat, 64) for feat in sparse_features] + [DenseFeat(feat, 1,) for feat in dense_features]
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(
    linear_feature_columns+ dnn_feature_columns)

train_model_input={}
for sec in sparse_features:
    train_model_input[sec]=train_gbdt_feat[:,int(sec)]
for i,sec in enumerate(dense_features):
    train_model_input[sec]=x_train[:,dense_pos[i]]

In [69]:
model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns, task='binary',
               l2_reg_embedding=1e-5, device='cuda:1')

model.compile("adam", "binary_crossentropy",
              metrics=["binary_crossentropy", "auc"],)

In [72]:
model.fit(train_model_input, y_train,
          batch_size=256, epochs=50, validation_split=0, verbose=2)

cuda:1
Train on 4837532 samples, validate on 0 samples, 18897 steps per epoch


RuntimeError: reduce failed to synchronize: device-side assert triggered

In [ ]:
data_df = pd.read_csv('data_df.csv',index_col=0)

category_df = member_df[['user_id', 'category_A', 'category_B','category_C', 'category_D', 'category_E','gender', 'visit_freq']]

# category_df.columns = ['user_id','category_A_raw','category_B_raw','category_C_raw','category_D_raw','category_E_raw']
category_df.columns = [x+'_raw' if x !='user_id' else x for x in category_df.columns ]

data_df = pd.merge(data_df,category_df,on='user_id',how='left')

drop_columns = ['is_answer', 'question_id', 'user_id', 'topic_id','follow_topics', 'interest_topics']
train_secs = data_df.columns.drop(drop_columns)
category_columns = category_df.columns.drop('user_id')
dense_features = data_df.columns.drop(category_columns).drop(drop_columns)
sparse_features = category_columns
target = ['is_answer']

data_df[sparse_features] = data_df[sparse_features].fillna('-1', )
data_df[dense_features] = data_df[dense_features].fillna(0, )

for feat in sparse_features:
    lbe = LabelEncoder()
    data_df[feat] = lbe.fit_transform(data_df[feat])

# 2.count #unique features for each sparse field,and record dense feature field name

fixlen_feature_columns = [SparseFeat(feat, data_df[feat].nunique())
                          for feat in sparse_features] + [DenseFeat(feat, 1,)
                                                          for feat in dense_features]

dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(
    linear_feature_columns+ dnn_feature_columns)

trian_start = 3838; train_end = 3867
train = data_df[train_secs.tolist()+['is_answer']][:train_df.shape[0]][train_df['invite_create_day']>train_end-7]

train_model_input = {name:train[name] for name in feature_names}

model = wdl.WDL(linear_feature_columns=dnn_feature_columns, dnn_feature_columns=dnn_feature_columns, task='binary',
               l2_reg_embedding=1e-5, device='cuda:1')
model.compile("adam", "binary_crossentropy",
              metrics=["binary_crossentropy", "auc"],)
model.fit(train_model_input, train[target].values,
          batch_size=512, epochs=35, validation_split=0, verbose=2)

model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns, task='binary',
               l2_reg_embedding=1e-5, device='cuda:1')

model.compile("adam", "binary_crossentropy",
              metrics=["binary_crossentropy", "auc"],)
model.fit(train_model_input, train[target].values,
          batch_size=256, epochs=50, validation_split=0, verbose=2)

test = data_df[train_secs.tolist()+['is_answer']][train_df.shape[0]:].reset_index(drop=True)

test_model_input = {name:test[name] for name in feature_names}

pred_ans = model.predict(test_model_input, 256)
print("")

In [ ]:
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))

In [ ]:
test_df['is_answer'] = pred_ans
test_df.to_csv('result.txt', index=False, header=False, sep='\t')

## 分析

In [ ]:
# mean
for sec in numeric_sectors:
    print(data_df[[sec,'is_answer']].groupby('is_answer').mean())

In [ ]:
days_sec = [x for x in data_df_copy.columns if 'day' in x]

In [ ]:
data_df['invite_answer']

In [ ]:
# 0
for sec in numeric_sectors:
    print(data_df[[sec,'is_answer']].groupby('is_answer').mean())

In [ ]:
# 加入num_answer_nums影响不大，加入word_count_question会显著导致过拟合

In [ ]:
data_df[numeric_sectors].head()

In [ ]:
trian_start = 3838; train_end = 3867
test_start = 3868; test_end = 3874
ans_start = 3807; ans_end = 3867
ques_start = 753;qus_end = 3874
/home/sc1/zk/pytorch-CycleGAN-and-pix2pix-master/results/cuhk_per_big/test_latest/images

In [ ]:
# data_df[numeric_sectors] = ans_data.fillna(0,axis=0)
# data_df[numeric_sectors] = ans_data.fillna(mean_values,axis=0)
# question_bins = [-1,0,1,3,500]
# member_bins = [-1,0,1,5,1000]
# data_df['question_answer_nums'] =pd.cut(data_df['question_answer_nums'],member_bins,labels=[1,2,3,4]).astype('int')
# data_df['user_answer_nums'] =pd.cut(data_df['user_answer_nums'],member_bins,labels=[1,2,3,4]).astype('int')

In [ ]:
# from lightgbm import LGBMClassifier
# model_lgb = LGBMClassifier(boosting_type='gbdt', num_leaves=64, learning_rate=0.01, n_estimators=2000,
#                            max_bin=425, subsample_for_bin=50000, objective='binary', min_split_gain=0,
#                            min_child_weight=5, min_child_samples=10, subsample=0.8, subsample_freq=1,
#                            colsample_bytree=1, reg_alpha=3, reg_lambda=5, seed=1000, n_jobs=-1, silent=True,
#                            num_threads = 30)
# model_lgb.fit(x_train, y_train, 
#                   eval_names=['train'],
#                   eval_metric=['logloss','auc'],
#                   eval_set=[(x_train, y_train)],
#                   early_stopping_rounds=3)

# y_pred = model_lgb.predict_proba(x_test)[:, 1]
# test_df['is_answer'] = y_pred
# test_df.to_csv('result.txt', index=False, header=False, sep='\t')

In [ ]:
for sec in ques_vecs_secs:
    print(data_df[[sec,'is_answer']].groupby('is_answer').mean())